In [1]:
from __future__ import annotations
import os
import pandas as pd
from config_tickers import SINGLE_TICKER
from config import DATA_SAVE_DIR
from config import START_DATE
from config import END_DATE
from config import TRADE_COLUMNS
from config import QUOTE_COLUMNS

### Helper Functions

In [2]:
def check_and_make_directories(directories: list[str]):
    paths = []
    for dir in directories:
        full_path = os.path.abspath(dir)

        if not os.path.exists("./" + dir):
            # print (dir)
            os.makedirs("./" + dir)
        paths.append(full_path)
    
    return paths

### Example Unified Data Processor

In [3]:
from torch_models.metadata.processor import DataProcessor

paths = check_and_make_directories(
    [DATA_SAVE_DIR])

kwargs = {
    "data_path": paths[0]
}

processor = DataProcessor(data_source='taq_clickhouse', data_path=paths[0])

# processor.download_data(
#     ticker=SINGLE_TICKER,
#     start_date=START_DATE,
#     end_date=END_DATE,
#     trade_cols=TRADE_COLUMNS,
#     quote_cols=QUOTE_COLUMNS
# )

# load from existing files
trades_df = pd.read_csv("sample_data/trades_AAPL_train.csv.gz", index_col=0,compression='gzip',nrows=1000)
quotes_df = pd.read_csv("sample_data/quotes_AAPL_train.csv.gz", index_col=0,compression='gzip',nrows=1000)

df = processor.preprocess(trades_df, quotes_df)

params = {
    "mode": "calendar",
    "prediction_length": 5, #measured in seconds
    "deltas": [(0, 1.)] #measured in seconds
}

features_df = processor.gen_taq_features(df, **params)

/home/jbohne/sbu/taq_db/TAQ-Query-Scripts/torch_models/metadata/data_processors/processor_taq.py:126: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  invalid_trades_idx.extend(new_trades[new_trades["Trade_Price"] <= 0].index)


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [18]:
quotes_df


,Time,Exchange,Symbol,Bid_Price,Bid_Size,Offer_Price,Offer_Size,Quote_Condition,Sequence_Number,National_BBO_Indicator,...,Short_Sale_Restriction_Indicator,LULD_BBO_Indicator,SIP_Generated_Message_Identifier,NBBO_LULD_Indicator,Participant_Timestamp,FINRA_ADF_Timestamp,FINRA_ADF_Market_Participant_Quote_Indicator,Security_Status_Indicator,Date,YearMonth
0,2022-01-03 03:59:00.091564,Z,AAPL,0.00,0.0,0.00,0.0,L,1252,1,...,0,NaN,NaN,NaN,35900088741000,NaN,NaN,,2022-01-03,202201
1,2022-01-03 03:59:00.198749,K,AAPL,0.00,0.0,0.00,0.0,L,2235,1,...,0,NaN,NaN,NaN,35900197520000,NaN,NaN,,2022-01-03,202201
2,2022-01-03 04:00:00.028174,Q,AAPL,177.92,4.0,0.00,0.0,R,4138,2,...,0,NaN,NaN,NaN,40000028160898,NaN,NaN,,2022-01-03,202201
3,2022-01-03 04:00:00.028307,Q,AAPL,177.92,4.0,178.80,4.0,R,4140,4,...,0,NaN,NaN,NaN,40000028294451,NaN,NaN,,2022-01-03,202201
4,2022-01-03 04:00:00.087199,P,AAPL,130.00,1.0,0.00,0.0,R,5834,0,...,0,NaN,NaN,NaN,40000086819840,NaN,NaN,,2022-01-03,202201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2022-01-03 04:19:57.938762,P,AAPL,177.81,1.0,178.23,1.0,R,42570,0,...,0,NaN,NaN,NaN,41957938398464,NaN,NaN,,2022-01-03,202201
996,2022-01-03 04:19:58.941067,P,AAPL,177.81,1.0,177.99,3.0,R,42601,2,...,0,NaN,NaN,NaN,41958940704768,NaN,NaN,,2022-01-03,202201
997,2022-01-03 04:19:59.274448,P,AAPL,177.81,1.0,177.98,1.0,R,42607,2,...,0,NaN,NaN,NaN,41959274084864,NaN,NaN,,2022-01-03,202201
998,2022-01-03 04:20:35.310274,K,AAPL,177.75,1.0,179.00,1.0,R,43385,0,...,0,NaN,NaN,NaN,42035310072000,NaN,NaN,,2022-01-03,202201


In [4]:
pd.set_option('display.max_rows', None)
features_df.head(100)

,Return,Breath,Immediacy,VolumeAll,VolumeAvg,VolumeMax,Lambda,LobImbalance,TxnImbalance,PastReturn,QuotedSpread,EffectiveSpread
Participant_Timestamp,,,,,,,,,,,,
2020-01-06 09:15:03.498942,NaN,1,1.000000,1.0,1.000000,1.0,NaN,NaN,0.000000,NaN,NaN,0.0
2020-01-06 09:15:04.398262,0.000054,1,1.000000,1.0,1.000000,1.0,NaN,-0.846154,0.000000,-0.000068,0.000271,0.0
2020-01-06 09:15:04.480789,0.000054,2,0.500000,31.0,15.500000,30.0,NaN,-0.846154,0.000000,-0.000068,0.000271,0.0
2020-01-06 09:15:07.130415,0.000089,1,1.000000,67.0,67.000000,67.0,0.000000,-0.846154,0.000000,-0.000068,0.000271,0.0
2020-01-06 09:15:07.134778,-0.000010,2,0.500000,200.0,100.000000,133.0,0.000150,-0.615385,0.665000,0.000000,0.000407,2.252951959291528e-05
2020-01-06 09:15:07.134778,-0.000010,2,0.500000,200.0,100.000000,133.0,0.000150,-0.615385,0.665000,0.000000,0.000407,2.252951959291528e-05
2020-01-06 09:15:08.110946,-0.000015,3,0.333333,201.0,67.000000,133.0,0.000149,-0.569231,0.666667,-0.000011,0.000434,2.2585982003711035e-05
2020-01-06 09:15:08.110946,-0.000021,3,0.333333,201.0,67.000000,133.0,0.000149,-0.569231,0.666667,-0.000011,0.000434,2.2585982003711035e-05
2020-01-06 09:15:08.266000,-0.000021,2,0.500000,101.0,50.500000,100.0,0.000000,-0.500000,-0.980198,0.000102,0.000474,0.00023516617157211972


### Example Torch Model ~ Lasso Model

Synthetic Linear Data

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from torch_models.metadata.pandas import PandasDataset
from torch_models.torch.lasso.estimator import LassoEstimator

In [4]:
np.random.seed(42)
x = np.arange(1, 10001)
y = 2*x + np.random.normal(0, 0.2, size=x.size)
df = pd.DataFrame(x, columns=['Feature1'])
df['Target'] = y
df.head()

,Feature1,Target
0,1,2.099343
1,2,3.972347
2,3,6.129538
3,4,8.304606
4,5,9.953169


In [5]:
train_size = int(0.8*len(df))
df_train, df_test = df.iloc[:train_size], df.iloc[train_size:]
train_set = PandasDataset(
    dataframes = df_train,
    target = 'Target',
    start_date="2020-01-06"
)
test_set = PandasDataset(
    dataframes = df_test,
    target = 'Target',
    start_date="2020-01-07"
)

train_kwargs = {
    "max_epochs" : 200
}

lasso_estimator = LassoEstimator(
    n_features = 1,
    l1_strength = 0.01,
    lr = 0.02,
    weight_decay = 0,
    trainer_kwargs = train_kwargs
)

In [6]:
lasso_predictor = lasso_estimator.train(
    train_data=train_set,
)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/zilinchen/anaconda3/envs/env_taq/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name  | Type                 | Params
-----------------------------------------------
0 | model | LassoRegressionModel | 2     
-----------------------------------------------
2         Trainable params
0         Non-trainable params
2         Total params
0.000     Total estimated model pa

Epoch 0: 100%|██████████| 8/8 [00:01<00:00,  4.66it/s, v_num=7, train_loss=1.47e+8]

/Users/zilinchen/anaconda3/envs/env_taq/lib/python3.11/site-packages/torch/autograd/__init__.py:251: UserWarning: The operator 'aten::sgn.out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1702400235349/work/aten/src/ATen/mps/MPSFallback.mm:13.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
Epoch 0, global step 8: 'train_loss' reached 147399616.00000 (best 147399616.00000), saving model to '/Users/zilinchen/Desktop/TAQ/TAQ-Query-Scripts/lightning_logs/version_7/checkpoints/epoch=0-step=8.ckpt' as top 1


Epoch 1: 100%|██████████| 8/8 [00:00<00:00, 119.62it/s, v_num=7, train_loss=1.34e+8]

Epoch 1, global step 16: 'train_loss' reached 133630584.00000 (best 133630584.00000), saving model to '/Users/zilinchen/Desktop/TAQ/TAQ-Query-Scripts/lightning_logs/version_7/checkpoints/epoch=1-step=16.ckpt' as top 1


...

Epoch 198: 100%|██████████| 8/8 [00:00<00:00, 139.48it/s, v_num=7, train_loss=1.700]

Epoch 198, global step 1592: 'train_loss' reached 1.70117 (best 1.70117), saving model to '/Users/zilinchen/Desktop/TAQ/TAQ-Query-Scripts/lightning_logs/version_7/checkpoints/epoch=198-step=1592.ckpt' as top 1


Epoch 199: 100%|██████████| 8/8 [00:00<00:00, 135.37it/s, v_num=7, train_loss=1.700]

Epoch 199, global step 1600: 'train_loss' reached 1.70111 (best 1.70111), saving model to '/Users/zilinchen/Desktop/TAQ/TAQ-Query-Scripts/lightning_logs/version_7/checkpoints/epoch=199-step=1600.ckpt' as top 1
`Trainer.fit` stopped: `max_epochs=200` reached.


Epoch 199: 100%|██████████| 8/8 [00:00<00:00, 120.26it/s, v_num=7, train_loss=1.700]


In [7]:
lasso_predictor.backtest(test_set)

Predicting DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 73.18it/s]
Out of sample R2: 0.9999975801716255
